<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/Fragility_Index_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# @title
# Walsh-compliant Fragility Index (FI) for a 2×2 table, Google Colab ready
# + Fragility Quotient (FQ) and Modified-arm Fragility Quotient (MFQ)
#
# Source: Walsh M, Srinathan SK, McAuley DF, Mrkobrada M, Levine O, Ribic C, et al.
# The statistical significance of randomized controlled trial results is
# frequently fragile: a case for a Fragility Index.
# J Clin Epidemiol 2014;67:622–8.
# https://doi.org/10.1016/j.jclinepi.2013.10.019.
#
# Layout:
#   |  Events | Non-events |
# A |    a    |     b      |
# B |    c    |     d      |
# Fisher's exact test (two-sided), α = 0.05; p ≤ α = significant, p > α = non-significant.
# Toggles apply only within the single arm with FEWER events at baseline;
# if tied, choose arm with FEWER total cases; if still tied, either arm (we use A).
# A toggle is one within-arm outcome change: event↔non-event. Row totals fixed, columns change.
#
# Added metrics (per FRAGILITY METRICS definitions):
#   FQ  = FI / n_total
#   MFQ = FI / (2 * n_mod)   where n_mod = size of the modified arm at baseline
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
#
# Heston, T. F. (2025). Fragility Metrics Toolkit (Version 2.1.0) [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0
#

# Install SciPy if missing
try:
    import scipy
except ImportError:
    !pip -q install scipy

from scipy.stats import fisher_exact

ALPHA = 0.05

def fisher_p(a, b, c, d):
    _, p = fisher_exact([[a, b], [c, d]], alternative="two-sided")
    return p

def is_significant(p, alpha=ALPHA):
    return p <= alpha

def choose_arm(a, b, c, d):
    # Arm with fewer events; if tie, arm with fewer total; if still tie, A.
    if a < c:
        return 'A'
    if c < a:
        return 'B'
    totA, totB = a + b, c + d
    if totA < totB:
        return 'A'
    if totB < totA:
        return 'B'
    return 'A'  # identical results by symmetry; choose A deterministically

def toggle_once(a, b, c, d, arm, direction):
    """
    Apply one within-arm toggle.
    direction in {"up","down"}:
      up   = non-event -> event  (increase events in chosen arm)
      down = event     -> non-event (decrease events in chosen arm)
    Returns new counts or None if impossible (no available cells to flip).
    """
    if arm == 'A':
        if direction == 'up':
            if b <= 0: return None
            return a+1, b-1, c, d
        else:  # down
            if a <= 0: return None
            return a-1, b+1, c, d
    else:  # arm B
        if direction == 'up':
            if d <= 0: return None
            return a, b, c+1, d-1
        else:  # down
            if c <= 0: return None
            return a, b, c-1, d+1

def steps_to_cross(a, b, c, d, arm, direction, alpha=ALPHA, max_iter=10**6):
    """
    Repeatedly apply the same-direction toggles within the chosen arm
    until the significance state flips. Returns (steps, final_p) or (None, last_p) if impossible.
    """
    base_p = fisher_p(a, b, c, d)
    base_state = is_significant(base_p, alpha)
    steps = 0
    A,B,C,D = a,b,c,d
    for _ in range(max_iter):
        nxt = toggle_once(A,B,C,D, arm, direction)
        if nxt is None:
            return None, fisher_p(A,B,C,D)
        A,B,C,D = nxt
        steps += 1
        p_now = fisher_p(A,B,C,D)
        if is_significant(p_now, alpha) != base_state:
            return steps, p_now
    return None, fisher_p(A,B,C,D)

def compute_fi_walsh(a, b, c, d, alpha=ALPHA):
    # Validate
    for x in (a,b,c,d):
        if not (isinstance(x, int) and x >= 0):
            raise ValueError("Cells a,b,c,d must be non-negative integers.")
    base_p = fisher_p(a,b,c,d)
    base_state = "significant" if is_significant(base_p, alpha) else "non-significant"

    # Fix the arm per Walsh tie-breaks at baseline
    arm = choose_arm(a,b,c,d)

    # Try both monotone directions within that arm; pick the smaller step count
    up_res   = steps_to_cross(a,b,c,d, arm, "up",   alpha)
    down_res = steps_to_cross(a,b,c,d, arm, "down", alpha)

    candidates = []
    if up_res[0]   is not None: candidates.append(("non-events → events", up_res[0], up_res[1]))
    if down_res[0] is not None: candidates.append(("events → non-events", down_res[0], down_res[1]))

    n_total = a + b + c + d
    n_mod = (a + b) if arm == 'A' else (c + d)

    if not candidates:
        return {
            "FI": None,
            "FQ": None,
            "MFQ": None,
            "baseline_p": base_p,
            "baseline_state": base_state,
            "arm": arm,
            "direction": None,
            "n_mod": n_mod,
            "final_p": base_p,
            "note": "FI: not attainable under Walsh constraints (within one arm; row totals fixed)."
        }

    # Minimal steps wins; FI is a non-negative integer
    direction, steps, p_final = min(candidates, key=lambda x: x[1])
    target_state = "non-significant" if base_state == "significant" else "significant"

    FI = int(steps)
    FQ = (FI / n_total) if n_total > 0 else None
    MFQ = (FI / (2 * n_mod)) if n_mod > 0 else None

    return {
        "FI": FI,
        "FQ": FQ,
        "MFQ": MFQ,
        "baseline_p": base_p,
        "baseline_state": base_state,
        "target_state": target_state,
        "arm": arm,
        "direction": direction,
        "n_mod": n_mod,
        "final_p": p_final
    }

def main():
    print("Enter 2×2 table cells as integers.")
    a = int(input("a (Arm A events): ").strip())
    b = int(input("b (Arm A non-events): ").strip())
    c = int(input("c (Arm B events): ").strip())
    d = int(input("d (Arm B non-events): ").strip())

    res = compute_fi_walsh(a,b,c,d, alpha=ALPHA)

    print("\n--- Fragility Metrics (Walsh-compliant) ---")
    print(f"Baseline Fisher p = {res['baseline_p']:.6f} → {res['baseline_state']} (α={ALPHA})")
    print(f"Chosen arm per Walsh tie-breaks: {res['arm']}")
    if res["FI"] is None:
        print("FI: not attainable")
        print(res["note"])
        return
    print(f"Target state: {res['target_state']}")
    print(f"FI  = {res['FI']} toggle(s) (within {res['arm']}, {res['direction']})")
    print(f"Modified arm: {res['arm']}")
    print(f"Toggle direction: {res['direction']}")
    print(f"Arm size used for MFQ (n_mod): {res['n_mod']}")
    if res["FQ"] is not None:
        print(f"FQ  = {res['FQ']:.6f}")
    else:
        print("FQ  = N/A")
    if res["MFQ"] is not None:
        print(f"MFQ = {res['MFQ']:.6f}")
    else:
        print("MFQ = N/A")
    print(f"Final Fisher p = {res['final_p']:.6f}")

if __name__ == "__main__":
    main()


Enter 2×2 table cells as integers.
a (Arm A events): 31
b (Arm A non-events): 8
c (Arm B events): 31
d (Arm B non-events): 1

--- Fragility Metrics (Walsh-compliant) ---
Baseline Fisher p = 0.035166 → significant (α=0.05)
Chosen arm per Walsh tie-breaks: B
Target state: non-significant
FI  = 1 toggle(s) (within B, events → non-events)
Modified arm: B
Toggle direction: events → non-events
Arm size used for MFQ (n_mod): 32
FQ  = 0.014085
MFQ = 0.015625
Final Fisher p = 0.101541
